### importing 

In [20]:
%load_ext autoreload
%autoreload 2 
import pandas as pd
import numpy as np
import pdfplumber
import re
import nltk
from urllib.parse import urljoin, quote_plus, quote, urlencode
from urllib.request import Request, urlopen
from io import StringIO, BytesIO
from oc_tryinghouse import HouseCredit
from try_scrape_demand import DemandDeposit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# DemandDeposit(abanca[2]['link'],[4],'0170').output()

NameError: name 'abanca' is not defined

demand deposit 
- bankinter = 4
- abanca = 4
- bai = 4
- bic = 4,6
- ctt = 4
housing credit 
- bankinter = 8,9,10,11,12,13,14,15,16,17
- abanca = 6,7,8
- bai = no product
- bic = 7,8,9,10,12,13
- ctt = 6


In [3]:
abanca={1:{'link':'https://www.abanca.pt', 'page':[]}, 
        2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf', 'page':[6,7,8]},
        3:{'link':'https://storage.googleapis.com/bank_price_pdfs/1_all_products_210412130040.pdf', 'page':[6]}}
bankinter={1:{'link':'https://www.bankinter.pt', 'page':[]},
           2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf', 'page':[8,9,10,11,12,13,14,15,16,17]},
           3:{'link': 'https://storage.googleapis.com/bank_price_pdfs/2_all_products_210412130350.pdf', 'page':[], 'notes':'image file'}}
bic={1:{'link':'https://www.bancobic.ao' ,'page':[]},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0079_/0079_PRE_0.pdf', 'page':[7,8,9,10,12,13]},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/3_all_products_210412130840.pdf', 'page':[], 'notes':'wrong file'}}
bai={1:{'link':'https://www.bancobai.ao', 'page':[]},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0008_/0008_PRE.pdf', 'page':[], 'notes':'No housing credit products'},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/4_all_products_210412130952.pdf', 'page':[], 'notes':'No housing credit products'}}
ctt={1:{'link':'https://www.bancoctt.pt/', 'page':[]},
     2:{'link':'https://clientebancario.bportugal.pt/sites/default/files/precario/0193_/0193_PRE.pdf', 'page':[6]},
     3:{'link':'https://storage.googleapis.com/bank_price_pdfs/5_all_products_210412131014.pdf', 'page':[6]}}

### dictionary

In [4]:
house_credit_com = {'admin':['Comissões associadas a atos administrativos 4.1 Não realização da escritura',
                             'Alteração do local da escritura',
                             'Declarações de dívida',
                             'Mudança de regime de crédito',
                             'Declarações de dívida',
                             'Pedido de 2ª via de Caderneta Predial',
                             'Emissão de declarações não obrigatórias por lei',
                             'Emissão de 2ª vias de Declaração para efeitos de IRS – Urgente',
                             'Emissão de 2º vias de Declaração para efeitos de IRS',
                             'Emissão de 2ª vias de faturas',
                             'Declaração de Dívida para Fins Diversos',
                             'Declaração de Encargos com Prestações'],
                    'certificates':['Emolumentos do registo predial', 'registo predial',
                                    'Certidão permanente on-line'],
                    'debt_recovery':['Comissão de recuperação de valores em dívida', 'Prestação até 50.000 €',
                                    'Prestação > 50.000 €', 'Comissão de recuperação de valores em dívida',
                                    'Prestação > 50.000,00€', 'Prestação ≤ 50.000,00€'],
                    'displacement':['Comissão de deslocação', 'Até 100 Kms', '101 a 250 Kms', '> 250 Kms '],
                    'early_payment':['Comissão de reembolso antecipado parcial', 'Taxa fixa', 'Taxa variável', 
                                    'Taxa fixa', 'Comissão de reembolso antecipado total', 'Comissão de antecipação',
                                    '(pré.aviso 7 dias)', 'Comissão de compra antecipada', '(pré-aviso 10 dias)',
                                    'Comissão de Reembolso Antecipado Parcial',
                                    'Comissão de reembolso antecipado total'],
                    'evaluation':['Avaliação', 'Imóvel residencial', 
                                 'Garagens e arrecadações não anexas ao imóvel residencial', 'Avaliação do Imóvel'],
                    'formalization':['Comissão de formalização', 'Formalização'],
                    'process':['Processo', 'Abertura de Processo',
                              'Desistência ou não conclusão do processo por motivos imputáveis ao cliente'],
                    'inspections':['Vistorias', 'em caso de construção ou realização de obras'],
                    'reanalysis':['Reanálise'],
                    'settlement':['Comissão de Liquidação de Prestação', 'Liquidação de Prestação'],
                    'solicitors_notary':['Emolumentos notariais', 'Solicitadoria', 'Notiário'],
                    'statements':['Emissão de extratos de conta de empréstimos liquidados', 'extrato', 'extratos',
                                  'extrato de conta', 'extrato mensal'],
                    'taxes':['Imposto do Selo sobre concessão de crédito', 'imposto', 'imposto de selo', 'impostos'],
                    'termination':['Cessação da posição contratual', 'cessação', 'rescisão', 'encerramento']}

### make function of pedro run

#### big function

In [5]:
# abanca_f = HouseCredit(abanca[2]['link'], abanca[2]['page']).scrape_all()

TypeError: __init__() missing 1 required positional argument: 'id_bank'

In [6]:
# ctt_f = HouseCredit(ctt[2]['link'], ctt[2]['page']).scrape_all()

TypeError: __init__() missing 1 required positional argument: 'id_bank'

In [ ]:
bankinter_f = HouseCredit(bankinter[2]['link'], bankinter[2]['page']).scrape_all()

In [ ]:
bic_f = HouseCredit(bic[2]['link'], bic[2]['page']).scrape_all()

#### function to avid page section:

In [ ]:
# newby = {}
# for page in abanca_f:
#     for k in abanca_f[page]:
#         newby[k]=abanca_f[page][k]

### scraping Names

#### importing function needed

In [ ]:
def get_pdf(link):
        remote = urlopen(Request(link)).read()
        memory = BytesIO(remote)
        return memory
def getting_text(link, page_demand):
        file = get_pdf(link)
        file = pdfplumber.open(file)
        print('extract pdf content to text...')
        print()
        if len(page_demand) > 1 :
            joined_text = []
            for el in page_demand:
                page = file.pages[el]
                text = page.extract_text()
                joined_text.append(text)
            text ='             NEXT          '.join(joined_text)
            text = text.replace('Isento', '0,00')
            text = text.replace('n/a', '0,00')
            text = re.sub('--', str(np.nan), text)
            return text
        else:
            page = file.pages[page_demand[0]]
            text = page.extract_text()
            text = text.replace('Isento', '0,00')
            text = text.replace('n/a', '0,00')
            text = re.sub('--', str(np.nan), text)
            return text

In [ ]:
def tokenize(text):
    
    if len(nltk.sent_tokenize(text)) < 15:
        text = text.replace('\n','. ')
        # text = len_sentences(text)
        text = nltk.sent_tokenize(text)
        return text
    text = text.replace('\n',' ')
    # text = len_sentences(text)
    text = nltk.sent_tokenize(text)
    return text

In [ ]:
def n_account(text,sent):
        finals = []
        tokenized = sent
        for sentence in tokenized:
            if 'Crédito' in sentence or 'Habitação' in sentence:
                finals.append(sentence)
            
        return finals

In [ ]:
def names(values):
    search = 'Comissão'
    search2 = 'Crédito / Particulares'
    search3 ='Aplicável'
    search4 = 'Nota 1  CréditoHabitação'
    search5 = 'Incluem-se'
    product = 'hello world'
    regular = []
    if len(values) < 10:
        return values
    else:
        for value in values:
            value = value.replace('Crédito à habitação e outros créditos hipotecários (cont.)','')
            value = value.replace('Crédito à habitação e outros créditos hipotecários','')
            value = value.replace('(cont.)','')
            value =value.replace('Comissões Acresce  Outras  Euros  Valor  Em %   Imposto condições (Mín/Máx) Anual', '')
            value = value.replace('Outros Créditos Hipotecários', '')
            if search not in value and search2 not in value and search3 not in value and search4 not in value and search5 not in value:
                if len(value) > 5:
                    regular.append(value)
                    regular += [product]
    return regular
#         return regular

#### importing text and file

In [ ]:
abanca_text = getting_text(abanca[2]['link'], abanca[2]['page'])
bankinter_text = getting_text(bankinter[2]['link'], bankinter[2]['page'])
bic_text = getting_text(bic[2]['link'], bic[2]['page'])
ctt_text = getting_text(ctt[2]['link'], ctt[2]['page'])

In [ ]:
abanca_sent = tokenize(abanca_text)
bankinter_sent = tokenize(bankinter_text)
bic_sent= tokenize(bic_text)
ctt_sent = tokenize(ctt_text)

In [ ]:
# abanca_sent

### testing

##### this is just the n_account function (already enought for abanca, ctt)

In [ ]:
abanca_n = n_account(abanca_text, abanca_sent)
ctt_n = n_account(ctt_text, ctt_sent)
bankinter_n = n_account(bankinter_text, bankinter_sent)
bic_n = n_account(bic_text,bic_sent)

In [ ]:
bic_n

##### now try to solve for bankinter and bic

In [ ]:
abanca_names = names(abanca_n)
ctt_names = names(ctt_n)
bankinter_names = names(bankinter_n)
bic_names = names(bic_n)

In [ ]:
bic_names

### associating name and scraped prices

#### abanca

In [ ]:
abanca_n == abanca_names , ctt_n==ctt_names

In [ ]:
# abanca_n

In [ ]:
new_dict = {}
for k,v in abanca_f.items():
    for name in abanca_names:
        new_dict[name]=v

In [ ]:
new_dict

#### ctt

In [ ]:
# ctt_names

In [ ]:
def association(function, names):  
    new_dict = {}
    for k,v in function.items():
        for name in names:
            new_dict[name]=v
    return new_dict

In [ ]:
new_dict_ctt = association(ctt_f,ctt_names)
new_dict_ctt.pop('Banco CTT, S.A. Operações Crédito-Particulares - Pág.1/2')
new_dict_ctt

#### bic

In [ ]:
len(bic_f) , len(bic_names)

In [ ]:
bankinter_names

In [ ]:
for i,name in enumerate(bic_names):
    print(f' THIS IS THE {i+1} NAME:  {name} \n')


- ho barato malissimo ma sticazzi

In [ ]:
type(bic_names)

In [ ]:
product = 'Mútuo a obras e construções (Fora de comercialização)'
bic_names += [product]

In [ ]:
new_dict_bic = association(bic_f,bic_names)

In [ ]:
abanca_names

In [ ]:
result = []
for i,sentence in enumerate(bankinter_n):
    if 'Crédito à habitação e outros créditos hipotecários' in sentence:
#         if '(cont.)' not in sentence:
          result.append(' '.join([sentence,bankinter_n[i+1]]))
len(result)

In [ ]:
result

In [ ]:
len(HouseCredit(bankinter[2]['link'], bankinter[2]['page'],'0269').final()

In [10]:
abanca_out = HouseCredit(abanca[2]['link'], abanca[2]['page'],'0170').final()

output was called
opening url: https://clientebancario.bportugal.pt/sites/default/files/precario/0170_/0170_PRE.pdf...
extract pdf content to text...

tokenize text...
found these headers in parsed pages: ['OPERAÇÕES DE CRÉDITO (PARTICULARES)                                                                                 ÍNDICE  2.1 Crédito à habitação e outros créditos hipotecários    Comissões  Acresce    Valor  Outras condições  Em %  Euros (Mín/Máx)  Imposto  anual  Produto: Crédito Habitação Regime Geral e Regime Deficientes.', 'Caracterização do produto: Crédito à habitação com garantia hipotecária destinado à aquisição, construção e realização de obras em  habitação própria e permanente, secundária ou para arrendamento (no Regime Deficientes só habitação própria e permanente) ou à  aquisição ou manutenção de direitos de propriedade sobre terrenos ou edifícios já existentes ou projetados.', 'OPERAÇÕES DE CRÉDITO (PARTICULARES)                                                      

In [16]:
len(abanca_out['subproducts'])
#     print(f'{abanca_out["subproducts"][sub]} \n')

3

In [26]:
bic_out = HouseCredit(bic[2]['link'], bic[2]['page'],'0079').output()

output was called
opening url: https://clientebancario.bportugal.pt/sites/default/files/precario/0269_/0269_PRE.pdf...
extract pdf content to text...

tokenize text...
/////////////// SCRAPE ALL /////////////////////
 {'page 8': {'Comissão de Estudo do Crédito Habitação': '260,00', 'Comissão de Avaliação': '220,00', 'Comissão de Formalização do Crédito Habitação': '120,00', 'Comissão de Contrato do Crédito Habitação': '90,00', 'Comissão de Tramitação': '90,00', 'Comissão de Fiança': '120,00', 'Comissão de Solicitadoria': '250,00', 'Comissão de Vistoria': '150,00'}, 'page 9': {'Comissão de Processamento de Prestação': '2,50', 'Comissão pela Recuperação de Valores em Dívida': '150,00', 'Comissão de Penalização por Amortização Antecipada Parcial no Crédito Habitação:': {'   - Taxa Variável': '0'}, 'Comissão de Penalização por Amortização Antecipada Total no Crédito Habitação:': {'   - Taxa Variável': '0'}}, 'page 10': {'Comissão de Estudo e Análise (Arrangement Fee)': '240,00', 'Comissão 

In [27]:
bankinter_out

{'subproducts': {'Crédito à habitação e outros créditos hipotecários - Crédito à habitação com ou sem garantia hipotecária para Residentes Legenda: Cont. (min/máx) Anual 1º Habitação para as Finalidades Aquisição, Aquisição e Obras (com garantia hipotecária), Obras (com garantia hipotecária), Construção, Aquisição de Terreno e Construção, Transferências; 2º Habitação para as Finalidades Aquisição, Aquisição e Obras (com garantia hipotecária), Obras (com garantia hipotecária), Construção, Transferências; Arrendamento para as Finalidades Aquisição, Aquisição e Obras (com garantia hipotecária), Obras (com garantia hipotecária), Transferências.': {'Comissão de Estudo do Crédito Habitação': '260,00',
   'Comissão de Avaliação': '220,00',
   'Comissão de Formalização do Crédito Habitação': '120,00',
   'Comissão de Contrato do Crédito Habitação': '90,00',
   'Comissão de Tramitação': '90,00',
   'Comissão de Fiança': '120,00',
   'Comissão de Solicitadoria': '250,00',
   'Comissão de Vistori